In [ ]:
# Date: 07.12.24
# Purpose: Copy of example Google notebook (function calls on data). 
# Theme: Tools/function use
# Status: Yet to start


In [ ]:
# Get it running then modify to use on csv.
# Ref: https://www.kaggle.com/code/timothys5/day-3-function-calling-with-the-gemini-api/edit

In [61]:
%pip install -q -U 'google-generativeai>=0.8.3'

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'google-generativeai": Expected package name at the start of dependency specifier
    'google-generativeai
    ^


In [ ]:
import google.generativeai as genai

C:\Users\Tim_S\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create a local database
For this minimal example, you'll create a local SQLite database and add some synthetic data so you have something to query.

Load the sql IPython extension so you can interact with the database using magic commands (the % instructions) to create a new, empty SQLite database

In [ ]:
# Installed
#pip install ipython-sql

In [133]:
%load_ext sql
%sql sqlite:///sample.db

# Don't think below required
# !pip install ipython-sql
# !pip install sqlite3

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [134]:
# Create the tables and insert some synthetic data. Feel free to tweak this structure and data.

In [135]:
%%sql
/*- Create the 'products' table*/
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
);

/*- Create the 'staff' table*/
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

/*- Create the 'orders' table*/
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

/*- Insert data into the 'products' table*/
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
	('Digital Camera', 699.99),
	('Electric Standing Desk', 599.99),
	('Graphics Card', 499.99),
	('Projector', 499.99),
	('VR Headset', 399.99),
	('Noise-Canceling Headphones', 249.99),
	('Portable Monitor', 249.99),
	('Action Camera', 299.99),
	('Gaming Chair', 299.99),
	('Tablet', 299.99),
	('Drawing Tablet', 199.99),
	('Monitor', 199.99),
	('Smartwatch', 199.99),
	('Docking Station', 159.99),
	('Gaming Headset', 149.99),
	('Printer', 149.99),
	('Keyboard', 129.99),
	('Portable SSD', 129.99),
	('Streaming Microphone', 129.99),
	('Wi-Fi Router', 129.99),
	('Smart Home Hub', 119.99),
	('Scanner', 99.99),
	('Smart Speaker', 99.99),
	('Headphones', 89.99),
	('Ring Light', 89.99),
	('CPU Cooler', 79.99),
	('External Hard Drive', 79.99),
	('Power Bank', 59.99),
	('Green Screen', 49.99),
	('Laptop Bag', 49.99),
	('Tripod', 49.99),
	('Webcam', 49.99),
	('Desk Lamp', 39.99),
	('Wireless Mouse', 39.99),
	('Mouse', 29.99),
	('USB Hub', 24.99),
	('Lens Cleaner Kit', 19.99),
	('Surge Protector', 19.99),
	('Ergonomic Mouse Pad', 14.99),
	('HDMI Cable', 14.99),
	('Bluetooth Adapter', 9.99);

/*- Insert data into the 'staff' table*/
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

/*- Insert data into the 'orders' table*/
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
42 rows affected.
3 rows affected.
3 rows affected.


[]

### Define database functions
Function calling with Gemini API's Python SDK can be implemented by defining an OpenAPI schema that is passed to the model. Alternatively you can define Python functions and let the SDK inspect them to automatically define the schema. In this latter case, it's important that the functions are type annotated and have accurate docstrings that describe what the functions do - the model has no insight into the function body, so the docs function as the interface.

By providing three key pieces of functionality - listing tables, describing a table, and executing a query - the LLM (or even another user) will have the basic tools needed to understand and interrogate the database.

Start with a database connection that will be used across all of the functions.

In [136]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

In [137]:
# The first function will list all tables available in the database. Define it, and test it out to ensure it works.
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables


['products', 'sqlite_sequence', 'staff', 'orders']

In [138]:
# Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(' - DB CALL: describe_table')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("products")

 - DB CALL: describe_table


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10, 2)')]

In [139]:
# Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a SELECT query. Now provide that functionality, and test that it works.
def execute_query(sql: str) -> list[list[str]]:
    """Execute a SELECT statement, returning the results."""
    print(' - DB CALL: execute_query')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from products")
#execute_query("select first_name from staff whee first_nae = 'Alice'")

 - DB CALL: execute_query


[(1, 'Laptop', 799.99),
 (2, 'Keyboard', 129.99),
 (3, 'Mouse', 29.99),
 (4, 'Electric Standing Desk', 599.99),
 (5, 'Laptop', 799.99),
 (6, 'Digital Camera', 699.99),
 (7, 'Graphics Card', 499.99),
 (8, 'Projector', 499.99),
 (9, 'VR Headset', 399.99),
 (10, 'Noise-Canceling Headphones', 249.99),
 (11, 'Portable Monitor', 249.99),
 (12, 'Action Camera', 299.99),
 (13, 'Gaming Chair', 299.99),
 (14, 'Tablet', 299.99),
 (15, 'Drawing Tablet', 199.99),
 (16, 'Monitor', 199.99),
 (17, 'Smartwatch', 199.99),
 (18, 'Docking Station', 159.99),
 (19, 'Gaming Headset', 149.99),
 (20, 'Printer', 149.99),
 (21, 'Keyboard', 129.99),
 (22, 'Portable SSD', 129.99),
 (23, 'Streaming Microphone', 129.99),
 (24, 'Wi-Fi Router', 129.99),
 (25, 'Smart Home Hub', 119.99),
 (26, 'Scanner', 99.99),
 (27, 'Smart Speaker', 99.99),
 (28, 'Headphones', 89.99),
 (29, 'Ring Light', 89.99),
 (30, 'CPU Cooler', 79.99),
 (31, 'External Hard Drive', 79.99),
 (32, 'Power Bank', 59.99),
 (33, 'Green Screen', 49.99),
 

## Implement function calls

Now you can put it all together in a call to the Gemini API.

Function calling works by adding specific messages to a chat session. When function schemas are defined and made available to the model and a conversation is started, instead of returning a text response, the model may return a `function_call` instead. When this happens, the client must respond with a `function_response`, indicating the result of the call, and the conversation can continue on as normal.

This function calling interaction normally happens manually, allowing you, the client, to validate and initiate the call. However the Python SDK also supports **automatic function calling**, where the supplied functions will be automatically invoked. This is a powerful feature and should only be exposed when it is safe to do so, such as when the functions have no [side-effects](https://en.wikipedia.org/wiki/Side_effect_(computer_science)).

Here's the state diagram representing the conversation flow with function calling. With automatic function calling, the bottom row is executed automatically by the Python SDK. In manual function calling, you write the code to run each step individually.

![function calling state diagram](https://codelabs.developers.google.com/static/codelabs/gemini-function-calling/img/gemini-function-calling-overview_1440.png)

In [ ]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database for a computer
store. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query."""


#model = genai.GenerativeModel("models/gemini-1.5-flash-latest", tools=db_tools, system_instruction=instruction)

genai.configure(api_key= )
# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
model = genai.GenerativeModel(model_name="gemini-1.5-flash", 
                              generation_config=generation_config, 
                              tools=db_tools, system_instruction=instruction
)


# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# Start a chat with automatic function calling enabled.
chat = model.start_chat(enable_automatic_function_calling=True)

In [141]:
# Now you can engage in a chat conversation where you can ask about the contents of the database.

In [143]:
#resp = chat.send_message("What is the cheapest product?", request_options=retry_policy)
#resp = chat.send_message("What is the most expensive product?", request_options=retry_policy)
resp = chat.send_message("Is there a staff members with the initials AS, if so what is their name?", request_options=retry_policy)
#resp = chat.send_message("Is there lamps for under $40?", request_options=retry_policy)
#resp = chat.send_message("Can you list the items that are available for 199.99?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
Yes, there are multiple staff members with the initials AS.  Their names are all Alice Smith.



In [144]:
# Inspecting the conversation
# To see the calls that the model makes, and what the client returns in response, you can inspect chat.history. 
# This helper function will print out each turn along with the relevant fields passed or returned.

import textwrap

def print_chat_turns(chat):
    """Prints out each turn in the chat history, including function calls and responses."""
    for event in chat.history:
        print(f"{event.role.capitalize()}:")

        for part in event.parts:
            if txt := part.text:
                print(f'  "{txt}"')
            elif fn := part.function_call:
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Function call: {fn.name}({args})")
            elif resp := part.function_response:
                print("  Function response:")
                print(textwrap.indent(str(resp), "    "))

        print()


print_chat_turns(chat)

User:
  "Do any staff members have the initials AS?"

Model:
  Function call: list_tables()

User:
  Function response:
    name: "list_tables"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              string_value: "products"
            }
            values {
              string_value: "sqlite_sequence"
            }
            values {
              string_value: "staff"
            }
            values {
              string_value: "orders"
            }
          }
        }
      }
    }


Model:
  Function call: describe_table(table_name=staff)

User:
  Function response:
    name: "describe_table"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              list_value {
                values {
                  string_value: "staff_id"
                }
                values {
                  string_value: "INTEGER"
                }
       

# Round 2. Checking if code can be extended to work with csv (in place of db)

In [214]:
# I think i figured it out. I need to create a python function that can accept the output of the llm. then it runs that code...

def execute_code(code: str):
    """
    Executes the provided Python code.

    Args:
        code (str): A string containing valid Python code.
    """
    try:
        exec(code)
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
code_to_run = """
for i in range(3):
    print(f"Hello, world! {i}")
"""
execute_code(code_to_run)


Hello, world! 0
Hello, world! 1
Hello, world! 2


In [184]:
import pandas as pd
# # Create a DataFrame
# data = {
#     "gender": ["Male", "Female", "Non-binary", "Female"],
#     "age": [25, 30, 22, 28],
#     "occupation": ["Engineer", "Doctor", "Artist", "Teacher"]
# }
# df = pd.DataFrame(data)
# display(df)

df = pd.read_csv('test.csv')


In [ ]:
# These are the Python functions defined above.
#db_tools = [list_tables, describe_table, execute_query]
# These are the Python functions defined above.


# These are the Python functions defined above.
# db_tools = [list_tables, describe_table, execute_query]
# using the tools
# available

instruction = """You are a helpful chatbot that can interact with an python dataframe for a computer
store. You will take the users questions and turn them into python queries.
Once you have the information you need, you will answer the user's question using
the data returned. There is only a signle dataframe to look at, and it hs the following columns {cols}
now issue a python query."""


#model = genai.GenerativeModel("models/gemini-1.5-flash-latest", tools=db_tools, system_instruction=instruction)

genai.configure(api_key= )
# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
model = genai.GenerativeModel(model_name="gemini-1.5-flash", 
                              generation_config=generation_config, 
                              tools=db_tools, system_instruction=instruction
)


# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# Start a chat with automatic function calling enabled.
chat = model.start_chat(enable_automatic_function_calling=True)
resp = chat.send_message("Are there more men or women in this data?", request_options=retry_policy)
print(resp.text)



 - DF CALL: execute_query


ValueError: Table 'df' does not exist.

In [197]:
#display(df.head(1))
df1 = df.groupby('Gender')['Gender'].count()
display(df1)

gender_counts = df['Gender'].value_counts()
display(gender_counts)

Gender
Female     70
Male      286
Name: Gender, dtype: int64

Gender
Male      286
Female     70
Name: count, dtype: int64

In [ ]:
# Pivoting to having Flash create the code, and python run the code
# These are the Python functions defined above.
#db_tools = [list_tables, describe_table, execute_query]
# These are the Python functions defined above.


import os
import google.generativeai as genai

genai.configure(api_key= )
# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 20,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)
chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message(instruction)
text = (response.text)


df = pd.read_csv('test.csv')
cols = df.columns.tolist()


# ------------- function -----
def answer_question_with_gemini(question, df):
    # Convert the DataFrame to a string for context
    context = df.to_string(index=False)
    # Generate a response using the Gemini model
    #response = model.generate_content(f'Answer the following question based on the data: {context}\n\nQuestion: {question}')
    #response = model.generate_content(f'Answer the following question (and show workings) based on the data: {context}\n\nQuestion: {question}')
    # response = model.generate_content(f"""You are a highly experienced analyst, who takes great pride in ensuring your answers are correct.
    #                                       You will be basing your answer on the dataframe with these columns {cols}
    #                                       Your task is to review the question {question} & then create python code that would answer that question.
    #                                       You will only provide the python code so it can be run without error.
    #                                       You will NOT provide and notes, explainations, details or other outputs that would cause the code to fail when attempting to
    #                                       """)
    response = model.generate_content(f"""You are an experienced data analayst who specialises writing python code to answer the questions.
             The data set you will be analysing contains the following columns {cols}
             You are to respond with the python code (referencing the columns names provided) that would answer the question.
             The code you provide is to be run a dataframe called df which already exists.
             Please do not include any output other than the code.
             The code will be surounded between these symbols # #
             """)
     
   

    return response.text

# ------------- question -----
# Example question
question = "Are there more males or females in the data?"  # Modify this based on your data columns

# Get the answer from Gemini
answer = answer_question_with_gemini(question, df)
print(f"Answer: {answer}")

# allowed_functions = {"print": print, "len": len}
# exec("print(len('Hello'))", {"__builtins__": None}, allowed_functions)



Answer: ```python
#
import pandas as pd

# Assuming 'df' is your existing DataFrame

# 1. What is the average income of loan applicants?
average_applicant_income = df['ApplicantIncome'].mean()
print(f"Average Applicant Income: {average_applicant_income}")


# 2. What is the distribution of loan amounts?
loan_amount_distribution = df['LoanAmount'].describe()
print(f"Loan Amount Distribution:\n{loan_amount_distribution}")


# 3. What is the relationship between credit history and loan approval (assuming a column indicating approval exists)?
#  This requires a column indicating loan approval (e.g., 'Loan_Status')
if 'Loan_Status' in df.columns:
    credit_history_loan_approval = pd.crosstab(df['Credit_History'], df['Loan_Status'])
    print(f"Credit History vs. Loan Approval:\n{credit_history_loan_approval}")


# 4. How many applicants are self-employed?
num_self_employed = df['Self_Employed'].value_counts()
print(f"Number of Self-Employed Applicants:\n{num_self_employed}")


# 5. What is

In [ ]:
exec()